<a href="https://colab.research.google.com/github/alicenie/comp4462_project/blob/main/Rescue_Priority.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rescue Priority Calculation

---

By Yankun Zhao

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Libraries

In [ ]:
import os
import re
import csv
import json
import copy
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
# pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import pprint
from tqdm.notebook import tqdm
import math
from sklearn.preprocessing import MinMaxScaler

# Data Preprocessing

In [ ]:
intensity_data = pd.read_csv("/content/drive/MyDrive/COMP4462/merge_10h_i.csv")
intensity_data.info()
intensity_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4556 entries, 0 to 4555
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         4556 non-null   int64  
 1   index              4556 non-null   object 
 2   sewer_and_water    4556 non-null   float64
 3   power              4556 non-null   float64
 4   roads_and_bridges  4556 non-null   float64
 5   medical            4556 non-null   float64
 6   buildings          4556 non-null   float64
 7   shake_intensity    4556 non-null   float64
 8   location           4556 non-null   int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 320.5+ KB


,Unnamed: 0,index,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,0,2020-04-06 00:00:00,5.636364,4.848485,4.878788,6.030303,6.151515,0.5,1
1,1,2020-04-06 00:30:00,2.000000,9.000000,9.000000,6.000000,0.000000,-1.0,1
2,2,2020-04-06 01:00:00,1.000000,4.000000,4.000000,8.000000,8.000000,0.0,1
3,3,2020-04-06 01:30:00,8.000000,8.666667,8.000000,3.333333,5.333333,1.0,1
4,4,2020-04-06 02:00:00,4.000000,2.666667,5.333333,7.000000,6.000000,-1.0,1


In [ ]:
uncertainty_data = pd.read_csv("/content/drive/MyDrive/COMP4462/merge_10h_u.csv")
uncertainty_data.info()
uncertainty_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4556 entries, 0 to 4555
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         4556 non-null   int64  
 1   index              4556 non-null   object 
 2   sewer_and_water    4556 non-null   float64
 3   power              4556 non-null   float64
 4   roads_and_bridges  4556 non-null   float64
 5   medical            4556 non-null   float64
 6   buildings          4556 non-null   float64
 7   shake_intensity    4556 non-null   float64
 8   location           4556 non-null   int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 320.5+ KB


,Unnamed: 0,index,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,0,2020-04-06 00:00:00,3.248251,3.709121,3.551035,3.196530,3.447111,0.57735,0
1,1,2020-04-06 00:30:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,1
2,2,2020-04-06 01:00:00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,1
3,3,2020-04-06 01:30:00,3.464102,1.527525,1.732051,4.932883,5.033223,-1.00000,0
4,4,2020-04-06 02:00:00,2.645751,3.785939,4.725816,3.000000,5.196152,-1.00000,0


In [ ]:
all_i = intensity_data[['index','sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity','location']].copy(deep=True)
all_i = all_i.rename(columns={"index": "start_time", "sewer_and_water": "sewer_and_water_i", "power": "power_i", "roads_and_bridges": "roads_and_bridges_i", "medical": "medical_i", "buildings": "buildings_i", "shake_intensity": "shake_intensity_i"})
all_i = all_i[['start_time', 'location', 'sewer_and_water_i', 'power_i', 'roads_and_bridges_i', 'medical_i', 'buildings_i', 'shake_intensity_i']]
all_i

,start_time,location,sewer_and_water_i,power_i,roads_and_bridges_i,medical_i,buildings_i,shake_intensity_i
0,2020-04-06 00:00:00,1,5.636364,4.848485,4.878788,6.030303,6.151515,0.5
1,2020-04-06 00:30:00,1,2.000000,9.000000,9.000000,6.000000,0.000000,-1.0
2,2020-04-06 01:00:00,1,1.000000,4.000000,4.000000,8.000000,8.000000,0.0
3,2020-04-06 01:30:00,1,8.000000,8.666667,8.000000,3.333333,5.333333,1.0
4,2020-04-06 02:00:00,1,4.000000,2.666667,5.333333,7.000000,6.000000,-1.0
...,...,...,...,...,...,...,...,...
4551,2020-04-10 21:00:00,19,5.625000,4.500000,6.500000,-1.000000,6.250000,1.0
4552,2020-04-10 21:30:00,19,6.000000,4.285714,6.142857,-1.000000,6.285714,1.0
4553,2020-04-10 22:00:00,19,5.400000,5.200000,6.600000,-1.000000,5.600000,1.0
4554,2020-04-10 22:30:00,19,5.000000,5.000000,8.250000,-1.000000,4.500000,1.0


In [ ]:
all_u = uncertainty_data[['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity']].copy(deep=True)
all_u = all_u.rename(columns={"sewer_and_water": "sewer_and_water_u", "power": "power_u", "roads_and_bridges": "roads_and_bridges_u", "medical": "medical_u", "buildings": "buildings_u", "shake_intensity": "shake_intensity_u"})
all_u

,sewer_and_water_u,power_u,roads_and_bridges_u,medical_u,buildings_u,shake_intensity_u
0,3.248251,3.709121,3.551035,3.196530,3.447111,0.57735
1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000
2,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000
3,3.464102,1.527525,1.732051,4.932883,5.033223,-1.00000
4,2.645751,3.785939,4.725816,3.000000,5.196152,-1.00000
...,...,...,...,...,...,...
4551,3.292307,2.976095,3.251373,-1.000000,3.412163,-1.00000
4552,3.366502,3.147183,3.338092,-1.000000,3.683942,-1.00000
4553,3.507136,3.033150,3.714835,-1.000000,4.277850,-1.00000
4554,3.915780,3.464102,0.500000,-1.000000,4.041452,-1.00000


## Merge intensity and uncertainty

In [ ]:
all_iu = pd.concat([all_i, all_u], axis='columns')
all_iu.info()
all_iu.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4556 entries, 0 to 4555
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   start_time           4556 non-null   object 
 1   location             4556 non-null   int64  
 2   sewer_and_water_i    4556 non-null   float64
 3   power_i              4556 non-null   float64
 4   roads_and_bridges_i  4556 non-null   float64
 5   medical_i            4556 non-null   float64
 6   buildings_i          4556 non-null   float64
 7   shake_intensity_i    4556 non-null   float64
 8   sewer_and_water_u    4556 non-null   float64
 9   power_u              4556 non-null   float64
 10  roads_and_bridges_u  4556 non-null   float64
 11  medical_u            4556 non-null   float64
 12  buildings_u          4556 non-null   float64
 13  shake_intensity_u    4556 non-null   float64
dtypes: float64(12), int64(1), object(1)
memory usage: 498.4+ KB


,start_time,location,sewer_and_water_i,power_i,roads_and_bridges_i,medical_i,buildings_i,shake_intensity_i,sewer_and_water_u,power_u,roads_and_bridges_u,medical_u,buildings_u,shake_intensity_u
0,2020-04-06 00:00:00,1,5.636364,4.848485,4.878788,6.030303,6.151515,0.5,3.248251,3.709121,3.551035,3.196530,3.447111,0.57735
1,2020-04-06 00:30:00,1,2.000000,9.000000,9.000000,6.000000,0.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000
2,2020-04-06 01:00:00,1,1.000000,4.000000,4.000000,8.000000,8.000000,0.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000
3,2020-04-06 01:30:00,1,8.000000,8.666667,8.000000,3.333333,5.333333,1.0,3.464102,1.527525,1.732051,4.932883,5.033223,-1.00000
4,2020-04-06 02:00:00,1,4.000000,2.666667,5.333333,7.000000,6.000000,-1.0,2.645751,3.785939,4.725816,3.000000,5.196152,-1.00000


In [ ]:
all_iu['start_time'] = pd.to_datetime(all_iu['start_time'], format="%Y-%m-%d %H:%M:%S")
all_iu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4556 entries, 0 to 4555
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   start_time           4556 non-null   datetime64[ns]
 1   location             4556 non-null   int64         
 2   sewer_and_water_i    4556 non-null   float64       
 3   power_i              4556 non-null   float64       
 4   roads_and_bridges_i  4556 non-null   float64       
 5   medical_i            4556 non-null   float64       
 6   buildings_i          4556 non-null   float64       
 7   shake_intensity_i    4556 non-null   float64       
 8   sewer_and_water_u    4556 non-null   float64       
 9   power_u              4556 non-null   float64       
 10  roads_and_bridges_u  4556 non-null   float64       
 11  medical_u            4556 non-null   float64       
 12  buildings_u          4556 non-null   float64       
 13  shake_intensity_u    4556 non-nul

# Rescue Priority Calculation

In [ ]:
damages = ["sewer_and_water", "power", "roads_and_bridges", "medical", "buildings", "shake_intensity"]
var_lambda = 1
rpv_list = []

for idx, row in all_iu.iterrows():
    rpv = 0 # Rescue Priority Value
    valid = False

    for d in damages:
        col_i = f"{d}_i"
        col_u = f"{d}_u"
        if row[col_i] >= 0 and row[col_u] >= 0:
            valid = True
            ivwi = row[col_i]/math.exp(var_lambda * row[col_u]) # Inverse-Variance Weighted Intensity
            rpv = rpv + ivwi
            # print(rpv)

    if valid:
        rpv_list.append(rpv)
    else:
        rpv_list.append(-1)


In [ ]:
all_rpv = pd.DataFrame(data = {'start_time': all_iu['start_time'], 'location': all_iu['location'], 'priority_value': rpv_list})
all_rpv.info()
all_rpv

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4556 entries, 0 to 4555
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   start_time      4556 non-null   datetime64[ns]
 1   location        4556 non-null   int64         
 2   priority_value  4556 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 106.9 KB


,start_time,location,priority_value
0,2020-04-06 00:00:00,1,1.200911
1,2020-04-06 00:30:00,1,-1.000000
2,2020-04-06 01:00:00,1,-1.000000
3,2020-04-06 01:30:00,1,3.605851
4,2020-04-06 02:00:00,1,0.773317
...,...,...,...
4551,2020-04-10 21:00:00,19,0.896280
4552,2020-04-10 21:30:00,19,0.767252
4553,2020-04-10 22:00:00,19,0.650816
4554,2020-04-10 22:30:00,19,5.339082


In [ ]:
minmax_scaler = MinMaxScaler()
intervals = []

for t in all_rpv['start_time'].value_counts().index:
    one_interval = all_rpv[all_rpv['start_time'] == t].copy(deep=True).sort_values(by=['priority_value'], ascending=False).reset_index(drop=True)
    num_rows = len(one_interval)
    num_valid_rows = len(one_interval[one_interval['priority_value'] >= 0])
    valid_pv = np.array(one_interval['priority_value'].tolist()[:num_valid_rows]).reshape(-1, 1)
    valid_pv = minmax_scaler.fit_transform(valid_pv)
    new_priority_value_col = valid_pv.ravel().tolist() + [-1 for i in range(num_rows-num_valid_rows)]
    one_interval['priority_value'] = new_priority_value_col

    priority = []
    for idx, row in one_interval.iterrows():
        if row['priority_value'] >= 0:
            priority.append(idx+1)
        else:
            priority.append(-1)
    one_interval['priority'] = priority
#     break
# one_interval
    intervals.append(one_interval)

all_priority = pd.concat(intervals).sort_values(by=['location', 'start_time'], ascending=True).reset_index(drop=True)
all_priority

,start_time,location,priority_value,priority
0,2020-04-06 00:00:00,1,0.591456,8
1,2020-04-06 00:30:00,1,-1.000000,-1
2,2020-04-06 01:00:00,1,-1.000000,-1
3,2020-04-06 01:30:00,1,0.179278,3
4,2020-04-06 02:00:00,1,0.064343,8
...,...,...,...,...
4551,2020-04-10 21:00:00,19,0.158693,15
4552,2020-04-10 21:30:00,19,0.060382,17
4553,2020-04-10 22:00:00,19,0.000000,18
4554,2020-04-10 22:30:00,19,0.688780,2


# Save

In [ ]:
output_path = "/content/drive/MyDrive/COMP4462/priority_map.csv"
all_priority.to_csv(output_path, index=False)